In [2]:
path = '/usr/local/bin/chromedriver-mac-x64/chromedriver'
import pandas as pd
import torch as tt
import re as re


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.1 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/austingrant/CS/TWOW/.venv/lib/python3.12/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/austingrant/CS/TWOW/.venv/lib/python3.12/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/austingrant/CS/TWOW/.venv/lib/python3.12/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_lo

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

# Set up Chrome options
chrome_options = Options()
# Uncomment to see the browser window
# chrome_options.add_argument("--headless")

# Path to your ChromeDriver executable
service = Service(path)  # Update this path

# Initialize WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)

def extract_post_details(url):
    driver.get(url)
    data = []
    try:
        # Wait for the content to load
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.content'))
        )
        
        # Find all posts
        posts = driver.find_elements(By.CSS_SELECTOR, '.content .post-main')
        print(f"Total posts found: {len(posts)}")
        
        for i, post in enumerate(posts):
            try:
                # Wait for the title element to be present
                WebDriverWait(post, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'h1 a'))
                )
                title_element = post.find_element(By.CSS_SELECTOR, 'h1 a')
                title = title_element.text
                
                # Wait for the date element to be present
                WebDriverWait(post, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, '.thedate'))
                )
                date_element = post.find_element(By.CSS_SELECTOR, '.thedate')
                date = date_element.text
                
                # Wait for content paragraphs to be present
                WebDriverWait(post, 10).until(
                    EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.post p'))
                )
                content_elements = post.find_elements(By.CSS_SELECTOR, '.post p')
                content = ' '.join([p.text for p in content_elements])
                
                # Print extracted data
                data.append({
                'Title': title,
                'Date': date,
                'Content': content
                })
            except Exception as e:
                print(f"Error extracting details for post {i+1}: {e}")
        return data

    except Exception as e:
        print(f"Error loading page: {e}")

def get_next_page_url():
    try:
        next_button = driver.find_element(By.CSS_SELECTOR, '.alignright a')
        return next_button.get_attribute('href')
    except Exception as e:
        print(f"Error finding next page button: {e}")
        return None

# URL of the first page
url = 'https://georgerrmartin.com/notablog/page/1/'  # Update as needed
page_number = 1
data = []
while url:
    print(f"Extracting posts from page {page_number}...")
    data.extend(extract_post_details(url))
    url = get_next_page_url()
    page_number += 1

# Close the WebDriver
driver.quit()
df = pd.DataFrame(data)
df.to_csv('grrm_notablog_posts.csv', index=False)


In [6]:
df = pd.read_csv('grrm_notablog_posts.csv')

In [7]:
pat = r'\w{3,9} \d{1,2}, \d{4} AT \d{1,2}:\d{2} ((AM)|(PM))'
df_filtered = df[df['Date'].str.contains(pat, regex=True, na=False)]
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
print(df.head())

/var/folders/bf/4d30byvn2vl0pld30w3h7sm80000gn/T/ipykernel_1551/3903705049.py:2: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_filtered = df[df['Date'].str.contains(pat, regex=True, na=False)]
/var/folders/bf/4d30byvn2vl0pld30w3h7sm80000gn/T/ipykernel_1551/3903705049.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Date'] = pd.to_datetime(df['Date'], errors='coerce')


                   Title                Date  \
0         September Morn 2024-09-03 11:54:00   
1    BURN HIM! BURN HIM! 2024-08-30 07:50:00   
2    Come to The Thicket 2024-08-29 12:02:00   
3     Remembering Howard 2024-08-22 09:29:00   
4  Here There Be Dragons 2024-07-11 07:06:00   

                                             Content  
0  My calendar is telling me it is now September....  
1  It is Fiesta here in Santa Fe, one of the olde...  
2  THE THICKET is one of Joe Lansdale’s best book...  
3  On June 29th, while I was off in Europe stumbl...  
4  I trust you all caught “The Red Dragon and the...  


In [ ]:
books = {
    "A Game of Thrones" : 
}